In [51]:
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px
import pandas as pd, numpy as np
pd.set_option('display.max_columns', None)
import sqlite3, sqlalchemy 

In [52]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 
seasons = ['2015_16', '2016_17', '2017_18', '2018_19', '2019_20', '2020_21', '2021_22', '2022_23', '2023_24']
dfs = [] 

for i in range(len(seasons)):
    df = pd.read_sql('master_'+seasons[i], con=engine) 
    df['Season'] = seasons[i].replace('_', '-')
    dfs.append(df) 

def convert_numeric(column): 
    return pd.to_numeric(column.astype(str).str.replace('%', ''))

master = (pd.concat(dfs, axis=0)
          .rename(columns={'TRB':'RPG', 'AST':'APG', 'PTS':'PPG'}) 
)

master[['PG%','SG%', 'SF%', 'PF%', 'C%']] = (master[['PG%','SG%', 'SF%', 'PF%', 'C%']]
                                            .fillna(0)
                                            .apply(convert_numeric)
)

master['3P%']

0      0.388
1      0.000
2        NaN
3      0.382
4      0.000
       ...  
567    0.143
568    0.373
569    0.208
570    0.333
571      NaN
Name: 3P%, Length: 4817, dtype: float64

In [63]:
def new_mean(x):
    return round(x.mean(numeric_only=True, skipna=True), 3)

temp = master.groupby('Season').apply(lambda x: x.apply(new_mean)).reset_index() 
temp['Player'] = 'League Average'
temp['Pos'] = 'All' 
temp['Tm'] = 'All' 
temp 

TypeError: Series.mean does not allow numeric_only=True with non-numeric dtypes.

In [54]:

app = Dash() 

app.layout = [
    html.Div(children=[dcc.RadioItems(options=['PPG', 'APG', 'RPG', '3P%', 'FG%', 'FT%', 'STL', 'BLK', 'TOV', 'PF', 'G', 'MP', 'DRtg', 'ORtg', 'WS', 'DWS'], value='PPG', id='controls', inline=True)], style={'text-align':'center', 'font-family':'calibri'}), 
    html.Div(children=[dcc.Dropdown(master.Player.unique(), value='LeBron James', id='dropdown-selection')], style={'font-family':'calibri'}), 
    html.Div( children=[dcc.Graph(figure={}, id='graph-content')]) 
]

@callback(
    Output(component_id='graph-content', component_property='figure'),
    Input('controls', 'value'), Input('dropdown-selection', 'value')
)
def update_graph(stat, player): 
    player = master[master.Player==player] 
    fig = px.line(player, x='Season', y=stat, color_discrete_sequence=['orange'], markers=True)
    return fig 

if __name__ == '__main__':
    app.run(debug=True)

NoLayoutException: Layout must be a dash component or a function that returns a dash component.